In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('/kaggle/input/training-and-ensemble')
sys.path.append('/kaggle/input/zip-file')

In [ ]:
import numpy as np
import pandas as pd
import torch
# Import các thư viện cần thiết
from sklearn.model_selection import train_test_split
# Import các lớp tùy chỉnh của bạn
from SVCSpecific import create_specific_svc_pipeline 
from SingleModel import SingleModel
from EnsembleModel import EnsembleModel
from AudioDataset import AudioDataset, audio_collate_fn
from AudioFeatureExtractor import AudioFeatureExtractor
from audio_evaluation import AudioMisclassificationAnalyzer
from MisclassificationReporter import MisclassificationReporter

# Import các lớp/hàm từ thư viện Hugging Face
from transformers import AutoFeatureExtractor, AutoModel
# Import joblib để lưu trữ hay nạp mô hình từ .pkl
import joblib

In [ ]:
# Đường dẫn gốc đến dữ liệu audio đã được phân loại vào các thư mục con
AUDIO_BASE_DIR = "/kaggle/input/data-speechclassification-labeling/DatasetLabeling/Audio_byClass" # THAY THẾ bằng đường dẫn THỰC TẾ của bạn

# Danh sách các tên lớp (tương ứng với tên thư mục con trong AUDIO_BASE_DIR)
CLASS_NAMES = ["Anger", "Happiness", "Sadness", "Neutral state"] # THAY THẾ bằng danh sách lớp THỰC TẾ của bạn

HF_MODEL_NAME_Wavlm = "microsoft/wavlm-large" 
HF_MODEL_NAME_Wav2vec = "facebook/wav2vec2-large-robust" 
HF_MODEL_NAME_Hubert = "facebook/hubert-large-ll60k" 
HF_MODEL_NAME_unispeech = "microsoft/unispeech-sat-large-sv" 
HF_MODEL_NAME_nest ="nvidia/ssl_en_nest_large_v1.0"
HF_MODEL_NAME_canary = "nvidia/canary-1b-flash" 

# Đường dẫn để lưu/nạp danh sách
AUDIO_LIST_CSV_PATH = "audio_files_list.csv"
# EXTRACTED_FEATURES_CSV_PATH = f"features_{HF_MODEL_NAME_Wavlm.replace('/', '_')}.csv"

# Đường dẫn file CSV cho báo cáo misclassification tổng hợp
REPORT_BASE_DIR = "summary_report.csv"

BATCH_SIZE = 16

ensemble_predictions_dict = {} # Dictionary để lưu dự đoán từ các mô hình/chiến lược cho báo cáo so sánh
list_of_aud_hub_wav = []
list_of_aud_hub = []
list_of_hub_wav = []

In [ ]:
print("--- Bắt đầu quy trình ---")

print("\nBước 1: Chuẩn bị danh sách file audio (toàn bộ dataset).")
all_audio_files_labels = []
all_audio_files_labels = AudioDataset.collect_audio_files(AUDIO_BASE_DIR, CLASS_NAMES)
if all_audio_files_labels:
    temp_dataset = AudioDataset(audio_files_list=all_audio_files_labels)
    temp_dataset.save_to_csv(AUDIO_LIST_CSV_PATH)
    del temp_dataset
print(f"Tổng số file audio thu thập được: {len(all_audio_files_labels)}")
# Tạo AudioDataset cho TOÀN BỘ dataset
full_dataset = AudioDataset(audio_files_list=all_audio_files_labels)
print(f"Full dataset có {len(full_dataset)} file.")

### Wavlm

In [ ]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Wavlm.csv" # Sửa tên file để phân biệt

feature_extractor = AudioFeatureExtractor(
    processor_name_or_path=HF_MODEL_NAME_Wavlm,
    model_name_or_path=HF_MODEL_NAME_Wavlm
)
# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = feature_extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    AudioFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Wavlm = np.array([item[0] for item in extracted_full_data])
    labels_list_Wavlm = [item[1] for item in extracted_full_data]
    filepaths_list_Wavlm = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del feature_extractor
    
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


In [ ]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Wavlm.shape}, Nhãn: {len(labels_list_Wavlm)}, Filepaths: {len(filepaths_list_Wavlm)}")
assert features_np_Wavlm.shape[0] == len(labels_list_Wavlm) == len(filepaths_list_Wavlm), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Wavlm)}")

In [ ]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Wavlm, labels_list_Wavlm, filepaths_list_Wavlm, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")

In [ ]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Wavlm_svc = SingleModel(
    name="Wavlm SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wavlm_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wavlm_svc.evaluate()
list_of_aud_hub_wav.append(model_Wavlm_svc)
list_of_hub_wav.append(model_Wavlm_svc)
if model_Wavlm_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wavlm_svc.get_name()] = model_Wavlm_svc.get_predictions()

In [ ]:
joblib.dump(model_Wavlm_svc,"model_Wavlm_svc.pkl")

### Hubert

In [ ]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Hubert.csv" # Sửa tên file để phân biệt

feature_extractor = AudioFeatureExtractor(
    processor_name_or_path=HF_MODEL_NAME_Hubert,
    model_name_or_path=HF_MODEL_NAME_Hubert
)
# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = feature_extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    AudioFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Hubert = np.array([item[0] for item in extracted_full_data])
    labels_list_Hubert = [item[1] for item in extracted_full_data]
    filepaths_list_Hubert = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del feature_extractor
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


In [ ]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Hubert.shape}, Nhãn: {len(labels_list_Hubert)}, Filepaths: {len(filepaths_list_Hubert)}")
assert features_np_Hubert.shape[0] == len(labels_list_Hubert) == len(filepaths_list_Hubert), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Hubert)}")

In [ ]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Hubert, labels_list_Hubert, filepaths_list_Hubert, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")

In [ ]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Hubert_svc = SingleModel(
    name="Hubert SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Hubert_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Hubert_svc.evaluate()
list_of_aud_hub_wav.append(model_Hubert_svc)
list_of_hub_wav.append(model_Hubert_svc)
list_of_aud_hub.append(model_Hubert_svc)
if model_Hubert_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Hubert_svc.get_name()] = model_Hubert_svc.get_predictions()

In [ ]:
joblib.dump(model_Hubert_svc,"model_Hubert_svc.pkl")

### Wav2vec

In [ ]:
print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
EXTRACTED_FEATURES_CSV_PATH = f"features_with_path_HF_MODEL_NAME_Wav2vec.csv" # Sửa tên file để phân biệt

feature_extractor = AudioFeatureExtractor(
    processor_name_or_path=HF_MODEL_NAME_Wav2vec,
    model_name_or_path=HF_MODEL_NAME_Wav2vec
)
# extract_dataset_embeddings giờ trả về list (np_array, label, filepath)
extracted_full_data = feature_extractor.extract_dataset_embeddings(
    dataset=full_dataset, # Trích xuất cho full dataset
    batch_size=BATCH_SIZE
)

if extracted_full_data:
    print(f"\nHoàn thành trích xuất cho full dataset. Lưu đặc trưng và metadata vào {EXTRACTED_FEATURES_CSV_PATH}.")
    # save_features_to_csv giờ nhận list (np_array, label, filepath)
    AudioFeatureExtractor.save_features_to_csv(extracted_full_data, EXTRACTED_FEATURES_CSV_PATH)

    # Tách kết quả trích xuất thành numpy arrays và list
    features_np_Wav2vec = np.array([item[0] for item in extracted_full_data])
    labels_list_Wav2vec = [item[1] for item in extracted_full_data]
    filepaths_list_Wav2vec = [item[2] for item in extracted_full_data] # Lấy list filepaths từ kết quả trích xuất
del feature_extractor
print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")

In [ ]:
print("\n--- Đặc trưng (toàn bộ dataset) đã sẵn sàng ---")
print(f"Shape đặc trưng: {features_np_Wav2vec.shape}, Nhãn: {len(labels_list_Wav2vec)}, Filepaths: {len(filepaths_list_Wav2vec)}")
assert features_np_Wav2vec.shape[0] == len(labels_list_Wav2vec) == len(filepaths_list_Wav2vec), "Số lượng mẫu không nhất quán sau khi chuẩn bị đặc trưng!"
print(f"Các lớp tìm thấy trong nhãn: {np.unique(labels_list_Wav2vec)}")

In [ ]:
print("\nBước 3: Chia dữ liệu (đặc trưng, nhãn, filepaths) thành tập train và test.")
# train_test_split CHỈ LÀM MỘT LẦN ở đây
X_train, X_test, y_train, y_test, train_filepaths, test_filepaths = train_test_split(
    features_np_Wav2vec, labels_list_Wav2vec, filepaths_list_Wav2vec, # Chia 3 thứ cùng lúc
    test_size=0.2,
    random_state=42,
)
print(f"Đã chia dữ liệu:")
print(f"  Train: X_train shape {X_train.shape}, y_train len {len(y_train)}, train_filepaths len {len(train_filepaths)}")
print(f"  Test: X_test shape {X_test.shape}, y_test len {len(y_test)}, test_filepaths len {len(test_filepaths)}")

In [ ]:
print("\nBước 4:## Chuẩn bị Mô hình SVC ##")
svc_pipeline = create_specific_svc_pipeline()
# Tạo đối tượng SingleModel cho SVC
model_Wav2vec_svc = SingleModel(
    name="Wav2vec SVM",
    model=svc_pipeline,
    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
    class_names=CLASS_NAMES
)
# Chạy pipeline (huấn luyện, predict, proba, uncertainty) cho mô hình đơn lẻ này
model_Wav2vec_svc.run_pipeline()
# Đánh giá kết quả mô hình đơn lẻ (Tùy chọn)
model_Wav2vec_svc.evaluate()
list_of_aud_hub_wav.append(model_Wav2vec_svc)
list_of_aud_hub.append(model_Wav2vec_svc)
if model_Wav2vec_svc.get_predictions() is not None:
    ensemble_predictions_dict[model_Wav2vec_svc.get_name()] = model_Wav2vec_svc.get_predictions()

In [ ]:
joblib.dump(model_Wav2vec_svc,"model_Wav2vec_svc.pkl")

### Ensemble

#### Audmodel + Hubert

In [ ]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble audmodel + hubert ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_aud_hub,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_aud_hub"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_aud_hub_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_aud_hub"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")

In [ ]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_aud_hub"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")

In [ ]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_aud_hub"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")

In [ ]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_aud_hub"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")

#### Hubert + Wavlm

In [ ]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble hubert + wavlm ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_hub_wav,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_hub_wav"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_hub_wav_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_hub_wav"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")

In [ ]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_hub_wav"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")

In [ ]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_hub_wav"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")

In [ ]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_hub_wav"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")

#### Audmodel + Hubert + Wavlm

In [ ]:
# --- Thực hiện Ensemble ---
print("\n--- Thực hiện và đánh giá Ensemble audmodel + hubert + wavlm ---")
# Truyền danh sách các mô hình đơn lẻ ĐÃ CHẠY pipeline, tên lớp và nhãn thật của tập test
ensemble_analyzer = EnsembleModel(
    models=list_of_aud_hub_wav,
    class_names=CLASS_NAMES,
    y_true=y_test # y_test chính là nhãn thật của tập kiểm tra
)

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Lowest (UL)")
y_pred_ul = ensemble_analyzer.predict_ul()
if y_pred_ul is not None:
    ensemble_predictions_dict["UL_Ensemble_aud_hub_wav"] = y_pred_ul
    ensemble_analyzer.evaluate_ensemble(y_pred_ul, strategy_name="Uncertainty Lowest (UL)")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Threshold (UT)")
# Cần xác định một ngưỡng cho UT. Đây là một siêu tham số có thể cần tinh chỉnh.
UT_THRESHOLD = 0.9
y_pred_ut = ensemble_analyzer.predict_ut(threshold=UT_THRESHOLD)
if y_pred_ut is not None:
    ensemble_predictions_dict[f"UT_Ensemble_aud_hub_wav_T{UT_THRESHOLD}"] = y_pred_ut
    ensemble_analyzer.evaluate_ensemble(y_pred_ut, strategy_name=f"Uncertainty Threshold (UT, threshold={UT_THRESHOLD})")

In [ ]:
print("\n# Đánh giá chiến lược Uncertainty Weighted (UW)")
y_pred_uw = ensemble_analyzer.predict_uw()
if y_pred_uw is not None:
    ensemble_predictions_dict["UW_Ensemble_aud_hub_wav"] = y_pred_uw
    ensemble_analyzer.evaluate_ensemble(y_pred_uw, strategy_name="Uncertainty Weighted (UW)")

In [ ]:
print("\n# Đánh giá chiến lược Confidence Weighted (CW)")
y_pred_cw = ensemble_analyzer.predict_cw()
if y_pred_cw is not None:
    ensemble_predictions_dict["CW_Ensemble_aud_hub_wav"] = y_pred_cw
    ensemble_analyzer.evaluate_ensemble(y_pred_cw, strategy_name="Confidence Weighted (CW)")

In [ ]:
print("\n# Đánh giá chiến lược Mean Probability Voting")
y_pred_mean_proba = ensemble_analyzer.predict_mean_proba_voting()
if y_pred_mean_proba is not None:
    ensemble_predictions_dict["MeanProba_Ensemble_aud_hub_wav"] = y_pred_mean_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_mean_proba, strategy_name="Mean Probability Voting")

In [ ]:
print("\n# Đánh giá chiến lược Max Probability Voting")
y_pred_max_proba = ensemble_analyzer.predict_max_proba_voting()
if y_pred_max_proba is not None:
    ensemble_predictions_dict["MaxProba_Ensemble_aud_hub_wav"] = y_pred_max_proba
    ensemble_analyzer.evaluate_ensemble(y_pred_max_proba, strategy_name="Max Probability Voting")

In [ ]:
print("\nToàn bộ quá trình hoàn thành!")

In [ ]:
# 7. Tạo báo cáo misclassification (Sử dụng MisclassificationReporter)
print("\nBước 7: Tạo báo cáo misclassification.")
reporter = MisclassificationReporter(base_output_dir=REPORT_BASE_DIR)

# --- Tạo báo cáo CSV CHỈ CHỨA MẪU SAI cho từng mô hình/chiến lược ---
print("\n--- Tạo báo cáo CSV chỉ chứa mẫu sai ---")
for name, y_pred in ensemble_predictions_dict.items():
    # Lấy nhãn thật (y_test) cho tất cả các dự đoán này
    # Đảm bảo y_pred và y_test có cùng thứ tự mẫu (được đảm bảo bởi workflow)
    reporter.generate_misclassified_csv(
            y_true=y_test, # Nhãn thật của tập test
            y_pred=y_pred, # Dự đoán của mô hình/chiến lược
            sample_identifiers=test_filepaths, # Đường dẫn file của tập test
            report_name=name # Tên báo cáo (tên mô hình/chiến lược)
    )

In [ ]:
# --- Tạo báo cáo CSV SO SÁNH chứa TẤT CẢ mẫu test ---
print("\n--- Tạo báo cáo CSV so sánh (tất cả mẫu test) ---")
comparison_report_filename = f"comparison_report_testset_svm.csv" # Tên file dựa trên mô hình nhúng
reporter.generate_comparison_report(
    sample_identifiers=test_filepaths, # Đường dẫn file của TẤT CẢ mẫu test
    y_true=y_test,                   # Nhãn thật của TẤT CẢ mẫu test
    predictions_dict=ensemble_predictions_dict, # Dictionary các dự đoán từ tất cả mô hình/chiến lược
    output_filename=comparison_report_filename
)

### Phát triển

#### Thử nghiệm thêm 3 mô hình mới 
<h4> unispeech,nest,canary </h4>

##### Unispeech

In [ ]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_unispeech,
#         model_name_or_path=HF_MODEL_NAME_unispeech
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


##### Nest

In [ ]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_nest,
#         model_name_or_path=HF_MODEL_NAME_nest
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")


##### Canary

In [ ]:
# print("\nBước 2: Đặc trưng chưa được trích xuất. Đang khởi tạo Feature Extractor và tiến hành trích xuất.")
# try:
#     # Khởi tạo AudioFeatureExtractor với mô hình và thiết bị đã chọn
#     feature_extractor = AudioFeatureExtractor(
#         processor_name_or_path=HF_MODEL_NAME_canary,
#         model_name_or_path=HF_MODEL_NAME_canary
#     )

#     # Trích xuất đặc trưng cho toàn bộ dataset
#     extracted_features_labels = feature_extractor.extract_dataset_embeddings(
#         dataset=dataset, # Sử dụng dataset đã chuẩn bị ở Bước 1
#         batch_size=BATCH_SIZE # Sử dụng batch size đã cấu hình
#     )
#     if extracted_features_labels:
#         print("\nBước 3: Hoàn thành trích xuất. Đang lưu đặc trưng vào CSV.")
#         # Lưu đặc trưng đã trích xuất vào CSV
#         feature_extractor.save_features_to_csv(extracted_features_labels, EXTRACTED_FEATURES_CSV_PATH)

#         # Chuyển kết quả trích xuất thành format numpy array và list labels
#         # để sẵn sàng cho bước tiếp theo (huấn luyện/kiểm tra)
#         features_np = np.array([item[0] for item in extracted_features_labels])
#         labels_list = [item[1] for item in extracted_features_labels]
#     else:
#         print("\nKhông có đặc trưng nào được trích xuất thành công. Không thể lưu hoặc tiếp tục.")
#         features_np = None
#         labels_list = None

#     del feature_extractor
# except Exception as e:
#     print(f"\nMột lỗi nghiêm trọng xảy ra trong quá trình trích xuất đặc trưng: {e}")
#     features_np = None
#     labels_list = None
# print("\n--- Kết thúc quy trình trích xuất đặc trưng ---")
